In [ ]:
# Softmax classifier for guessing minesweeper board position and whether it has a mine or not

In [1]:
# Import libraries for simulation
import tensorflow as tf
import numpy as np
import random as r
import datetime as dt
import multiprocessing as mp

/home/ruben/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dimensions = (8,8)
mineProbability = 0.16      # Probability that a square contain a mine

In [3]:
# Clears a square on the minesweeper board.
# If it had a mine, return true
# Otherwise if it has no adjacent mines, recursively run on adjacent squares
# Return false
def clearSquare(board,adjacency,row,col):
    rows,cols = dimensions
    if board[row,col] == 1:
        return True
    if adjacency[row,col] >= 0:
        return False
    n = 0
    for r in range(row-1,row+2):
        for c in range(col-1,col+2):
            if 0 <= r and r < rows and 0 <= c and c < cols:
                n += board[r,c]
    adjacency[row,col] = n
    if n == 0:
        for r in range(row-1,row+2):
            for c in range(col-1,col+2):
                if 0 <= r and r < rows and 0 <= c and c < cols:
                    clearSquare(board,adjacency,r,c)
    return False

In [4]:
# This takes a mine board and gives a mine count with mines removed, and other random squares removed
def boardPartialMineCounts(board):
    clearProbability = r.uniform(0.05,0.5)
    result = np.full(dimensions,-1)
    for index, x in np.random.permutation(list(np.ndenumerate(board))):
        row,col = index
        if not(x) and result[row,col] == -1 and r.uniform(0,1) < clearProbability:
            clearSquare(board,result,row,col)
    return result

In [5]:
# Generates a random training batch of size n
def randomBoard(i):
    return(np.random.random(dimensions) < mineProbability)

def encodeCountsOneHot(counts):
    countsOneHot = np.zeros((counts.size,10))
    countsOneHot[np.arange(counts.size), counts.flatten() + 1] = 1
    return(countsOneHot.flatten())

def validGuesses(boardAndCounts):
    board,counts = boardAndCounts
    validGuesses = np.append(((counts == -1).astype(int) - board).flatten().astype(float),
        board.flatten().astype(float))
    validGuessesSum = sum(validGuesses)
    if validGuessesSum > 0:
        return(validGuesses / validGuessesSum)
    else:
        return(np.zeros(board.size*2))

try:
    cpus = mp.cpu_count()
except NotImplementedError:
    cpus = 2   # arbitrary default

pool = mp.Pool(processes=cpus)

def next_training_batch(n):
    boards = pool.map(randomBoard, range(n))
    counts = pool.map(boardPartialMineCounts, boards)
    batch_xs = pool.map(encodeCountsOneHot, counts)
    batch_ys = pool.map(validGuesses, zip(boards,counts))
    return(batch_xs, batch_ys, boards)

Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Process ForkPoolWorker-4:
Process ForkPoolWorker-7:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bo

In [6]:
# Create the model
rows, cols = dimensions
size = rows*cols
mineCountsOneHot = tf.placeholder(tf.float32, [None, size*10], name="mineCountsOneHot")
W = tf.Variable(tf.truncated_normal([size*10, size], stddev=0.01), name="W")
b = tf.Variable(tf.truncated_normal([size], stddev=0.01), name="b")
y = tf.matmul(mineCountsOneHot, W) + b

In [7]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.01)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.01)
  return tf.Variable(initial)

In [8]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [9]:
y_image = tf.reshape(y, [-1, rows, cols, 1])

# First convolution layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(y_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Second convolution layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Third convolution layer
W_conv3 = weight_variable([5, 5, 64, 64])
b_conv3 = bias_variable([64])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)

# Fully connected layer
W_fc1 = weight_variable([64, 1024])
b_fc1 = bias_variable([1024])

h_pool3_flat = tf.reshape(h_pool3, [-1, 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Output layer
W_fc2 = weight_variable([1024, size*2])
b_fc2 = bias_variable([size*2])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [10]:
validGuessAverages = tf.placeholder(tf.float32, [None, size*2], name="validGuessAverages")

In [11]:
# Loss function
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=validGuessAverages, logits=y_conv))

In [12]:
# Counts the number of errors if we take the best guess for each sample
'''
def countErrors(xs,ys):
    predictions = sess.run(y_conv, feed_dict={mineCountsOneHot: batch_xs, keep_prob: 1.0})
    bestSquares = pool.map(np.argmax, predictions)
    unfrees = [(b == 0).astype(int) for b in batch_ys]
    frees = [unfrees[i][bestSquares[i]] for i in range(len(batch_xs))]
    return(sum(frees))
'''

'\ndef countErrors(xs,ys):\n    predictions = sess.run(y_conv, feed_dict={mineCountsOneHot: batch_xs, keep_prob: 1.0})\n    bestSquares = pool.map(np.argmax, predictions)\n    unfrees = [(b == 0).astype(int) for b in batch_ys]\n    frees = [unfrees[i][bestSquares[i]] for i in range(len(batch_xs))]\n    return(sum(frees))\n'

In [13]:
guesses = tf.diag_part(tf.gather(validGuessAverages,tf.argmax(y_conv,1), axis=1))
countErrors = tf.reduce_sum(tf.map_fn(lambda x: tf.cast(x < 1e-6, tf.int32), guesses, dtype=tf.int32))

In [14]:
# Summaries for tensorboard
_ = tf.summary.scalar('loss', cross_entropy)
_ = tf.summary.scalar('errors', countErrors)

In [15]:
# Optimiser
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [16]:
savePathPrefix = '/media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/' + str(rows) + 'x' + str(cols) + '/'

modelDataPath = savePathPrefix

def summaryPath(run):
    return(savePathPrefix + 'runs/' + str(run) + '/')

In [17]:
# Create session and initialise or restore stuff
saver = tf.train.Saver()

sess = tf.InteractiveSession()

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(summaryPath(6), sess.graph)

In [18]:
tf.global_variables_initializer().run()

In [19]:
# Restore model?
saver.restore(sess, modelDataPath + "model-20610")

INFO:tensorflow:Restoring parameters from /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-20610


In [20]:
# Train
for iteration in range(20611,1000001):
    batch_xs, batch_ys, _ = next_training_batch(10000)
    _ = sess.run(train_step, feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys, keep_prob: 0.5})
    loss, summary = sess.run([cross_entropy, merged], feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys, keep_prob: 1.0})
    writer.add_summary(summary, iteration)
    print('%s: Loss at step %s: %s' % (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), iteration, loss))
    if iteration % 10 == 0:
        save_path = saver.save(sess, modelDataPath + 'model', global_step=iteration)
        print("Model saved in file: %s" % save_path)

2017-11-10 05:20:31: Loss at step 20611: 3.82701
2017-11-10 05:20:34: Loss at step 20612: 3.82867
2017-11-10 05:20:37: Loss at step 20613: 3.8132
2017-11-10 05:20:40: Loss at step 20614: 3.80787
2017-11-10 05:20:43: Loss at step 20615: 3.81314
2017-11-10 05:20:46: Loss at step 20616: 3.8004
2017-11-10 05:20:49: Loss at step 20617: 3.81549
2017-11-10 05:20:52: Loss at step 20618: 3.80614
2017-11-10 05:20:55: Loss at step 20619: 3.80928
2017-11-10 05:20:58: Loss at step 20620: 3.81373
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-20620
2017-11-10 05:21:01: Loss at step 20621: 3.82117
2017-11-10 05:21:04: Loss at step 20622: 3.81299
2017-11-10 05:21:07: Loss at step 20623: 3.81784
2017-11-10 05:21:10: Loss at step 20624: 3.81022
2017-11-10 05:21:13: Loss at step 20625: 3.80756
2017-11-10 05:21:16: Loss at step 20626: 3.81005
2017-11-10 05:21:19: Loss at step 20627: 3.80442
2017-11-10 05:21:22: Loss at step 20628: 3.82551
2017-11-10 05:21:25: Loss

2017-11-10 05:27:35: Loss at step 20752: 3.79823
2017-11-10 05:27:38: Loss at step 20753: 3.79472
2017-11-10 05:27:41: Loss at step 20754: 3.81511
2017-11-10 05:27:44: Loss at step 20755: 3.80973
2017-11-10 05:27:47: Loss at step 20756: 3.80932
2017-11-10 05:27:50: Loss at step 20757: 3.81056
2017-11-10 05:27:52: Loss at step 20758: 3.81553
2017-11-10 05:27:55: Loss at step 20759: 3.81159
2017-11-10 05:27:58: Loss at step 20760: 3.80917
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-20760
2017-11-10 05:28:01: Loss at step 20761: 3.82324
2017-11-10 05:28:04: Loss at step 20762: 3.82263
2017-11-10 05:28:08: Loss at step 20763: 3.80032
2017-11-10 05:28:11: Loss at step 20764: 3.81442
2017-11-10 05:28:14: Loss at step 20765: 3.81827
2017-11-10 05:28:17: Loss at step 20766: 3.81848
2017-11-10 05:28:20: Loss at step 20767: 3.81798
2017-11-10 05:28:23: Loss at step 20768: 3.80676
2017-11-10 05:28:26: Loss at step 20769: 3.81909
2017-11-10 05:28:30: Lo

2017-11-10 05:34:34: Loss at step 20893: 3.80175
2017-11-10 05:34:37: Loss at step 20894: 3.80867
2017-11-10 05:34:40: Loss at step 20895: 3.8053
2017-11-10 05:34:43: Loss at step 20896: 3.8137
2017-11-10 05:34:46: Loss at step 20897: 3.81364
2017-11-10 05:34:49: Loss at step 20898: 3.81383
2017-11-10 05:34:52: Loss at step 20899: 3.81697
2017-11-10 05:34:54: Loss at step 20900: 3.8095
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-20900
2017-11-10 05:34:57: Loss at step 20901: 3.80523
2017-11-10 05:35:00: Loss at step 20902: 3.80491
2017-11-10 05:35:03: Loss at step 20903: 3.80916
2017-11-10 05:35:06: Loss at step 20904: 3.82444
2017-11-10 05:35:09: Loss at step 20905: 3.82352
2017-11-10 05:35:12: Loss at step 20906: 3.81723
2017-11-10 05:35:15: Loss at step 20907: 3.81297
2017-11-10 05:35:18: Loss at step 20908: 3.80768
2017-11-10 05:35:21: Loss at step 20909: 3.80791
2017-11-10 05:35:24: Loss at step 20910: 3.81115
Model saved in file: /medi

2017-11-10 05:41:26: Loss at step 21034: 3.81444
2017-11-10 05:41:29: Loss at step 21035: 3.80893
2017-11-10 05:41:32: Loss at step 21036: 3.8111
2017-11-10 05:41:35: Loss at step 21037: 3.82291
2017-11-10 05:41:38: Loss at step 21038: 3.81643
2017-11-10 05:41:41: Loss at step 21039: 3.81345
2017-11-10 05:41:44: Loss at step 21040: 3.83244
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-21040
2017-11-10 05:41:47: Loss at step 21041: 3.80984
2017-11-10 05:41:49: Loss at step 21042: 3.80654
2017-11-10 05:41:52: Loss at step 21043: 3.79697
2017-11-10 05:41:55: Loss at step 21044: 3.80954
2017-11-10 05:41:58: Loss at step 21045: 3.80897
2017-11-10 05:42:01: Loss at step 21046: 3.80085
2017-11-10 05:42:04: Loss at step 21047: 3.80341
2017-11-10 05:42:07: Loss at step 21048: 3.80114
2017-11-10 05:42:10: Loss at step 21049: 3.80689
2017-11-10 05:42:13: Loss at step 21050: 3.80818
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.M

2017-11-10 05:48:19: Loss at step 21175: 3.80523
2017-11-10 05:48:22: Loss at step 21176: 3.82239
2017-11-10 05:48:24: Loss at step 21177: 3.82791
2017-11-10 05:48:27: Loss at step 21178: 3.81191
2017-11-10 05:48:30: Loss at step 21179: 3.80767
2017-11-10 05:48:33: Loss at step 21180: 3.81262
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-21180
2017-11-10 05:48:36: Loss at step 21181: 3.81898
2017-11-10 05:48:39: Loss at step 21182: 3.80516
2017-11-10 05:48:42: Loss at step 21183: 3.8133
2017-11-10 05:48:45: Loss at step 21184: 3.80604
2017-11-10 05:48:48: Loss at step 21185: 3.80697
2017-11-10 05:48:51: Loss at step 21186: 3.80972
2017-11-10 05:48:54: Loss at step 21187: 3.82264
2017-11-10 05:48:57: Loss at step 21188: 3.81181
2017-11-10 05:49:00: Loss at step 21189: 3.80979
2017-11-10 05:49:03: Loss at step 21190: 3.80881
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-21190
2017-11-10 05:49:05: Loss a

...

2017-11-10 12:20:47: Loss at step 29211: 3.79881
2017-11-10 12:20:50: Loss at step 29212: 3.81048
2017-11-10 12:20:53: Loss at step 29213: 3.80453
2017-11-10 12:20:56: Loss at step 29214: 3.79132
2017-11-10 12:20:59: Loss at step 29215: 3.80447
2017-11-10 12:21:02: Loss at step 29216: 3.80446
2017-11-10 12:21:05: Loss at step 29217: 3.8035
2017-11-10 12:21:08: Loss at step 29218: 3.79798
2017-11-10 12:21:11: Loss at step 29219: 3.80822
2017-11-10 12:21:14: Loss at step 29220: 3.80581
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-29220
2017-11-10 12:21:17: Loss at step 29221: 3.80382
2017-11-10 12:21:20: Loss at step 29222: 3.80909
2017-11-10 12:21:23: Loss at step 29223: 3.79054
2017-11-10 12:21:26: Loss at step 29224: 3.80555
2017-11-10 12:21:29: Loss at step 29225: 3.813
2017-11-10 12:21:32: Loss at step 29226: 3.79824
2017-11-10 12:21:35: Loss at step 29227: 3.79961
2017-11-10 12:21:38: Loss at step 29228: 3.79355
2017-11-10 12:21:41: Loss 

2017-11-10 12:27:45: Loss at step 29352: 3.80049
2017-11-10 12:27:48: Loss at step 29353: 3.80498
2017-11-10 12:27:51: Loss at step 29354: 3.79946
2017-11-10 12:27:54: Loss at step 29355: 3.79834
2017-11-10 12:27:57: Loss at step 29356: 3.80385
2017-11-10 12:28:00: Loss at step 29357: 3.80273
2017-11-10 12:28:03: Loss at step 29358: 3.80461
2017-11-10 12:28:06: Loss at step 29359: 3.79307
2017-11-10 12:28:09: Loss at step 29360: 3.80028
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-29360
2017-11-10 12:28:12: Loss at step 29361: 3.79957
2017-11-10 12:28:15: Loss at step 29362: 3.79731
2017-11-10 12:28:18: Loss at step 29363: 3.79534
2017-11-10 12:28:21: Loss at step 29364: 3.79993
2017-11-10 12:28:24: Loss at step 29365: 3.79886
2017-11-10 12:28:27: Loss at step 29366: 3.79524
2017-11-10 12:28:30: Loss at step 29367: 3.79182
2017-11-10 12:28:33: Loss at step 29368: 3.80271
2017-11-10 12:28:36: Loss at step 29369: 3.80687
2017-11-10 12:28:39: Lo

2017-11-10 12:34:45: Loss at step 29493: 3.79458
2017-11-10 12:34:48: Loss at step 29494: 3.79184
2017-11-10 12:34:51: Loss at step 29495: 3.7983
2017-11-10 12:34:55: Loss at step 29496: 3.79848
2017-11-10 12:34:58: Loss at step 29497: 3.80652
2017-11-10 12:35:01: Loss at step 29498: 3.79502
2017-11-10 12:35:04: Loss at step 29499: 3.79893
2017-11-10 12:35:07: Loss at step 29500: 3.81014
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-29500
2017-11-10 12:35:10: Loss at step 29501: 3.8072
2017-11-10 12:35:13: Loss at step 29502: 3.8017
2017-11-10 12:35:16: Loss at step 29503: 3.79796
2017-11-10 12:35:18: Loss at step 29504: 3.80238
2017-11-10 12:35:21: Loss at step 29505: 3.79399
2017-11-10 12:35:24: Loss at step 29506: 3.80614
2017-11-10 12:35:27: Loss at step 29507: 3.80167
2017-11-10 12:35:30: Loss at step 29508: 3.7912
2017-11-10 12:35:33: Loss at step 29509: 3.80249
2017-11-10 12:35:36: Loss at step 29510: 3.79911
Model saved in file: /media

2017-11-10 12:41:45: Loss at step 29634: 3.7958
2017-11-10 12:41:48: Loss at step 29635: 3.79348
2017-11-10 12:41:51: Loss at step 29636: 3.79875
2017-11-10 12:41:54: Loss at step 29637: 3.79975
2017-11-10 12:41:57: Loss at step 29638: 3.79318
2017-11-10 12:42:00: Loss at step 29639: 3.80207
2017-11-10 12:42:03: Loss at step 29640: 3.81237
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-29640
2017-11-10 12:42:06: Loss at step 29641: 3.80103
2017-11-10 12:42:09: Loss at step 29642: 3.80841
2017-11-10 12:42:12: Loss at step 29643: 3.79532
2017-11-10 12:42:15: Loss at step 29644: 3.79961
2017-11-10 12:42:18: Loss at step 29645: 3.80038
2017-11-10 12:42:21: Loss at step 29646: 3.80336
2017-11-10 12:42:24: Loss at step 29647: 3.80381
2017-11-10 12:42:27: Loss at step 29648: 3.80459
2017-11-10 12:42:30: Loss at step 29649: 3.79518
2017-11-10 12:42:33: Loss at step 29650: 3.80453
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.M

KeyboardInterrupt: 

In [ ]:
# Test trained model on larger batch size
batch_xs, batch_ys, _ = next_training_batch(10000)
print(sess.run(cross_entropy, feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys, keep_prob: 1.0}))

In [ ]:
# Run a test
batchSize = 10000
batch_xs, batch_ys,_ = next_training_batch(batchSize)

#countErrors(batch_xs,batch_ys)
len(batch_xs)

In [ ]:
# Find boards that we failed on
batchSize = 1000
batch_xs, batch_ys, _ = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys})
bestSquares = [pred.argmax() for pred in predictions]
unfrees = (batch_ys == 0).astype(int)
guesses = [unfrees[i][bestSquares[i]] for i in range(batchSize)]
for i in range(batchSize):
    if guesses[i] == 1:
        print(batch_xs[i].reshape(dimensions))
        summary = sess.run(tf.summary.image('mine_miss', tf.reshape((batch_xs[i]+1).astype(float),[-1,rows,cols,1]), 100))
        writer.add_summary(summary)

In [ ]:
#batch_xs = [[-1,1,-1,0,0,0,-1,-1,-1,1,1,1,-1,-1,1,2,2,1,1,-1,2,1,1,-1,-1,2,2,-1,-1,2,-1,1,1,0,-1,-1,2,-1,-1,4,-1,2,-1,1,2,-1,1,0,1,2,-1,3,2,2,1,-1,2,-1,1,0,0,1,1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,0,0,3,-1,4,1,2,-1,1,-1,-1,0,0,0,2,-1,-1,-1,2,-1,1,0,0,0,-1,1,2,-1,2,1,2,2,3,3,2,-1,-1,1,-1,1,-1,0,1,2,-1,-1,-1,1,1,1,-1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,1,-1,-1,-1]]
batch_xs0 = [-1] * (size)
batch_xs0[0] = 1
batch_xs0[1] = 1
batch_xs0[cols] = 1

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCounts: [batch_xs0]})
bestSquares = [pred.argmax() for pred in predictions]

print(bestSquares[0] // cols, bestSquares[0] % cols)

In [ ]:
np.save("./W", sess.run(W))

In [ ]:
np.save("./b", sess.run(b))

In [ ]:
np.savez("./model", sess.run([W,b]))